In [1]:
import qiskit as qk
import numpy as np
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator, Pauli
from scipy.signal import convolve2d
from qiskit.circuit import Parameter
from sympy import *
from qiskit import IBMQ
import tkinter
import math
import os
%matplotlib inline

import matplotlib.pyplot as plt

In [2]:
def place_piece(turn, board, row, col):
    if turn % 2 == 0: board[row-1][col-1] = 'X'
    else: board[row-1][col-1] = 'O'
            
def idx(row, col, n): return n*row+col

def get_min_num_qubits(n):
    search_size = n**2
    num = 0
    while True:
        if search_size <= 2**num: return num
        num += 1
    
def is_power_of_2(n):
    return math.ceil(math.log10(n)/math.log10(2)) == math.floor(math.log10(n)/math.log10(2))

In [3]:
def end_state(board, num_connect, turn):
    board_cpy = [row[:] for row in board]
    for row in range(n):
        for col in range(n):
            if board_cpy[row][col] == 'X': board_cpy[row][col] = 1 if turn % 2 == 0 else 0
            if board_cpy[row][col] == 'O': board_cpy[row][col] = 1 if turn % 2 == 1 else 0
    horizontal_kernel = np.array([[1]*num_connect])
    vertical_kernel = np.transpose(horizontal_kernel)
    diag1_kernel = np.eye(num_connect)
    diag2_kernel = np.fliplr(diag1_kernel)
    detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]
    for kernel in detection_kernels:
        if (convolve2d(board_cpy, kernel, mode="valid") == num_connect).any():
            return True
    return False

def get_int_from_user(message="", lower_bound=1):
    loop_bool = True
    while loop_bool:
        try:
            ret = int(input(message))
            if lower_bound <= ret: loop_bool = False
            else: print("Invalid input: lower bound = {}".format(lower_bound))
        except ValueError: print("Could not read user input")
    return ret

a, b, c, d, e, g, h, i, j,k,l ,m,n,o,p,q,r,s,t,u,v,w,x,y,z= symbols('a b c d e g h i j k m l n o p q r s t u v w x y z')
alphabet= [a,b,c,d,e,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z]
init_printing(use_unicode=True)

In [4]:
def has_token(position, qc_positions, opp_positions):
    for it in qc_positions:
        if (position[0]==it[0]):
            if(position[1]==it[1]):
                return 1
    for it in opp_positions:
        if (position[0]==it[0]):
            if(position[1]==it[1]):
                return -1
    return 0

def change_of_base(position,n):
    qpos=[]
    rows=position[0]
    cols=position[1]
    for it in range(round(math.log(n,2))):
        #Rows
        if(rows>=(n/(2**(it+1)))):
            qpos.append(1)
            rows=rows-(n/(2**(it+1)))
        else:
            qpos.append(0)
        #columns
        if(cols>=(n/(2**(it+1)))):
            qpos.append(1)
            cols=cols-(n/(2**(it+1)))
        else:
            qpos.append(0)
    return qpos

In [5]:
def rev_change_of_base(bitstr, n):
    print(bitstr)
    col=0
    row=0
    counter=0
    pos_counter=2
    for it in bitstr:
        if(pos_counter%2==0):
            if (int(it)==1):
                col+= (2**(counter))
            pos_counter+=1
        else:
            if (int(it)==1):
                row+= (2**(counter))
            counter+=1
            pos_counter+=1
    return (row,col)

def writer(position,n):
    qpos = change_of_base(position,n)
    temp_func = 1
    counter = 0
    for sub_qpos in qpos:
            if sub_qpos==1: temp_func *= alphabet[counter]
            else: temp_func *= (1-alphabet[counter])
            counter+=1
    return temp_func

In [6]:
def Int_cost(position,qc_positions, opp_positions,n):
    inter_cost = 0
    #left edge
    if (position[1]==0):
        #top
        if(position[0]==0):
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
        #bottom
        elif(position[1]==n-1):
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]+1,qc_positions,opp_positions],n)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]+1],n)
        #everthing else
        else:
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
            # 2 BLOCKS
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]],n)
    #right edge
    elif(position[1]==n-1):
        #top
        if(position[0]==0):
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
        #bottom
        elif(position[1]==n-1):
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)
        #everthing else
        else:
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)   
            # 2 BLOCKS
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]],n)
    #everything else
    else:
        #top
        if(position[0]==0):
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
        #bottom
        elif(position[1]==n-1):
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]+1],n)
        #everthing else
        else:
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]-1],n)
            if (has_token([position[0]-1,position[1]-1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]-1],n)
            if (has_token([position[0]+1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]+1,position[1]+1],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)==0):
                inter_cost += writer([position[0]-1,position[1]+1],n)
            # 2 BLOCKS
            if (has_token([position[0]+1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]],n)
            if (has_token([position[0]-1,position[1]],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]],n)
            if (has_token([position[0],position[1]+1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0],position[1]-1],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0],position[1]+1],n)
            if (has_token([position[0],position[1]-1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0],position[1]+1],n)
            if (has_token([position[0]-1,position[1]+1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]+1,position[1]-1],n)
            if (has_token([position[0]+1,position[1]-1],qc_positions,opp_positions)!=0):
                inter_cost += 2*writer([position[0]-1,position[1]+1],n)
    return inter_cost

In [7]:
def cost_func(qc_positions, opp_positions, n, cost_function=0):
    #qc positions
    for pos in qc_positions:
        temp_func = writer(pos,n)
        #immediate positions
        int_func = Int_cost(pos,qc_positions,opp_positions,n)
        cost_function += -10*temp_func + int_func   
    #player positions
    for pos in opp_positions:
        temp_func = writer(pos,n)
        #immediate positions
        int_func = Int_cost(pos,qc_positions,opp_positions,n)
        cost_function += -20*temp_func + (3/4)*int_func  
    return expand(cost_function)

In [8]:
def get_contribution_to_hamiltonian(to_add, num_qubits):
    sym_map = {a:0, b:1, c:2, d:3, e:4, g:5, h:6, i:7, j:8, k:9, l:10, m:11, n:12, o:13, p:14, q:15, r:16, s:17, t:18, u:19, v:20, w:21, x:22, y:23, z:24}
    inter_str = str(to_add)
    const_idx = 0
    try: const_idx = inter_str.index("*")
    except ValueError: return 0 * Operator(Pauli('I' * num_qubits))
    try: coef = int(inter_str[:const_idx])
    except ValueError: coef = float(inter_str[:const_idx])
    monoms = list(to_add.free_symbols)
    lab = 'I' * num_qubits # monoms = [a, d, f]
    for idx in range(len(monoms)): lab = lab[:sym_map[monoms[idx]]] + 'Z' + lab[sym_map[monoms[idx]] + 1:]
    #len(monoms) = 1 -> local, >1 -> interaction 
    return coef * Operator(Pauli(lab))

In [9]:
def get_hamiltonian_from_cost_func(cost_func_e, num_qubits):
    to_add_list = list(Add.make_args(cost_func_e))
    H_c = Operator(Pauli(label = 'I' * num_qubits))
    for i in range(len(to_add_list)):
        H_c += get_contribution_to_hamiltonian(to_add_list[i], num_qubits)
    return H_c

def create_state_from_outcome(outcome):
    # Turns string of 0s and 1s into quantum state
    zero = np.array([[1],[0]])
    one = np.array([[0],[1]])
    
    state = np.array([[1]])
    for elem in outcome[::-1]: # reverse bit ordering to calculate state
        if elem == '0':
            state = np.kron(state, zero)
        else:
            state = np.kron(state, one)
    return state

def expectation_value(counts, shots, Hc):
    # Computes expectation value with respect to Hc
    hc_matr = Hc.data # change to a numpy array
    exp_val = 0
    for outcome in counts.keys():
        state = create_state_from_outcome(outcome)
        prob = counts[outcome]/shots
        exp_val += prob*(state.conjugate().transpose() @ hc_matr @ state)
    return np.real(exp_val[0,0])

In [10]:
def state_prep(qr,cr):
    circ = qk.QuantumCircuit(qr, cr)
    circ.h(qr)
    circ.barrier()
    return circ

In [56]:
def mixer(beta,qr,cr):
    circ = qk.QuantumCircuit(qr, cr)
    circ.rx(2*beta, qr)
    circ.barrier()
    return circ
def get_quantum_circ(to_add, num_qubits,beta,gamma,qr,cr):
    circ = qk.QuantumCircuit(qr, cr)
    sym_map = {a:0, b:1, c:2, d:3, e:4, g:5, h:6, i:7, j:8, k:9, l:10, m:11, n:12, o:13, p:14, q:15, r:16, s:17, t:18, u:19, v:20, w:21, x:22, y:23, z:24}
    inter_str = str(to_add)
    const_idx = 0
    try: const_idx = inter_str.index("*")
    except ValueError: return 0 * Operator(Pauli('I' * num_qubits))
    try: coef = int(inter_str[:const_idx])
    except ValueError: coef = float(inter_str[:const_idx])
    monoms = list(to_add.free_symbols)
    lab = 'I' * num_qubits # monoms = [a, d, f]
    for idx in range(len(monoms)): lab = lab[:sym_map[monoms[idx]]] + 'Z' + lab[sym_map[monoms[idx]] + 1:]
    #len(monoms) = 1 -> local, >1 -> interaction 
    temp= qk.QuantumCircuit(qr, cr)
    temp.append(Operator(coef*gamma*Pauli(label=lab)),qr)
    print(np.array(Pauli(label=lab)))
    print(lab)
    #temp= qk.transpile(temp, backend=Aer.get_backend('qasm_simulator'), basis_gates=['rz','id','cx','h'])
    temp.draw('mpl', fold=-1, filename="pure_panic")
    return  temp

In [62]:
def cost(gamma,num_qubits,qr):
    circ = qk.QuantumCircuit(qr)
    
    # Interaction terms
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i == j: continue # sum over i =/= j, skip i == j
            circ.cx(qr[i], qr[j])
            circ.rz(2*gamma/8, qr[j]) # J_{ij} = 1/8 for i != j
            circ.cx(qr[i], qr[j])
    circ.barrier()
    
    # Local terms
    circ.rz(2*2*gamma, qr) # h_i = 2        
    circ.barrier()
    return circ

def choose_best_pos(ham_op, cost_func_e, n):
    num_qubits= get_min_num_qubits(n)
    qr = qk.QuantumRegister(num_qubits)
    cr = qk.ClassicalRegister(num_qubits)
    qaoa_p1 = qk.QuantumCircuit(qr, cr)
    beta = Parameter('b')
    gamma = Parameter('g')
    
    to_add_list = list(Add.make_args(cost_func_e))
    qaoa_p1.compose(state_prep(qr,cr), inplace=True)
    #create cost function   #### didnt have enough time finished
    #for it in range(len(to_add_list)):
        #temp = get_quantum_circ(to_add_list[it], num_qubits,beta,gamma,qr,cr)
        #temp.draw('mpl', fold=-1, filename="pure_panic")
        #qaoa_p1.unitary(temp,qk,label=it)
        
    #mixer function an measures
    qaoa_p1.compose(cost(gamma,n,qr), inplace=True)
    qaoa_p1.compose(mixer(beta,qr,cr), inplace=True)
    qaoa_p1.measure(qr,cr)
    qaoa_p1.draw('mpl', fold=-1, filename="pure_panic")
    beta_val = 3*np.pi/5 # initial choice
    gamma_val = np.pi/2 # initial choice
    qaoa_p1_set = qaoa_p1.bind_parameters({beta: beta_val, gamma: gamma_val})
    shots = 500
    backend = Aer.get_backend('qasm_simulator')
    job = qk.execute(qaoa_p1_set, backend=backend, shots=shots)
    results = job.result()
    counts = results.get_counts(qaoa_p1_set)
    
    beta_list = 2*np.pi*np.linspace(0,1,10)
    gamma_list = np.pi*np.linspace(0,1,10)

    all_data = []
    for beta_val in beta_list:
        gamma_data = []
        for gamma_val in gamma_list:
            qaoa_p1_set = qaoa_p1.bind_parameters({beta: beta_val, gamma: gamma_val})
            job = qk.execute(qaoa_p1_set, backend=backend, shots=shots)
            results = job.result()
            counts = results.get_counts(qaoa_p1_set)
            exp_val = expectation_value(counts, shots, ham_op)
            gamma_data.append(exp_val)
        all_data.append(gamma_data)
        
    opt_val = np.min(all_data) # we want the max <H_C>
    for i in range(len(all_data)):
        for j in range(len(all_data[i])):
            if all_data[i][j] == opt_val:
                opt_i = i
                opt_j = j
                break
    opt_beta = beta_list[opt_i]
    opt_gamma = gamma_list[opt_j]
    
    qaoa_p1_set = qaoa_p1.bind_parameters({beta: beta_val, gamma: gamma_val})
    job = qk.execute(qaoa_p1_set, backend=backend, shots=shots)
    results = job.result()
    counts = results.get_counts(qaoa_p1_set)
    best_bitstr = max(counts, key = counts.get)
    plot_histogram(counts)
    row, col =rev_change_of_base(best_bitstr, n)
    row =int(row+1)
    col =int(col+1)
    return row,col,counts

In [63]:
import time

def game_loop():
    global n, board, human_pos, machine_pos
    human_pos = []
    machine_pos = []
    n = get_int_from_user(message="Board length/width (N x N sized board): ", lower_bound=2)
    while not is_power_of_2(n):
        print("N must be a power of 2! (eg. N = 2, 4, 8, etc.)")
        n = get_int_from_user(message="Board length/width (N x N sized board): ", lower_bound=2)
    num_qubits = get_min_num_qubits(n)
    num_connect = get_int_from_user(message="Number of blocks to connect to win (X in \"connect-X\"): ", lower_bound=3)
    board = [[0]*n for _ in range(n)]
    turn = 0
    
    while not end_state(board, num_connect, turn):
        print("Turn: {}".format("Human Player" if turn % 2 == 0 else "Machine Player"))
        print(np.array(board))
        if turn % 2 == 0: # Human turn
            row = get_int_from_user(message="Row Number [1 to {}, inclusive]: ".format(n))
            col = get_int_from_user(message="Column Number [1 to {}, inclusive]: ".format(n))
            human_pos.append([row, col])
        else: # Machine turn
            # get row, col from quantum algorithm
            cost = cost_func(machine_pos, human_pos, n)
            ham = get_hamiltonian_from_cost_func(cost, num_qubits)
            row, col, counts = choose_best_pos(ham, cost, n)
            print(row,col)
            plot_histogram(counts, filename="./PANIC.png")
            machine_pos.append([row, col])
            
        place_piece(turn, board, row, col)
        turn += 1 
        
game_loop()

Board length/width (N x N sized board):  4
Number of blocks to connect to win (X in "connect-X"):  3


Turn: Human Player
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


Row Number [1 to 4, inclusive]:  1
Column Number [1 to 4, inclusive]:  1


Turn: Machine Player
[['X' '0' '0' '0']
 ['0' '0' '0' '0']
 ['0' '0' '0' '0']
 ['0' '0' '0' '0']]
0111
4 3
Turn: Human Player
[['X' '0' '0' '0']
 ['0' '0' '0' '0']
 ['0' '0' '0' '0']
 ['0' '0' 'O' '0']]


Row Number [1 to 4, inclusive]:  1
Column Number [1 to 4, inclusive]:  2


Turn: Machine Player
[['X' 'X' '0' '0']
 ['0' '0' '0' '0']
 ['0' '0' '0' '0']
 ['0' '0' 'O' '0']]
1110
2 4
Turn: Human Player
[['X' 'X' '0' '0']
 ['0' '0' '0' 'O']
 ['0' '0' '0' '0']
 ['0' '0' 'O' '0']]


Row Number [1 to 4, inclusive]:  1
Column Number [1 to 4, inclusive]:  3


Turn: Machine Player
[['X' 'X' 'X' '0']
 ['0' '0' '0' 'O']
 ['0' '0' '0' '0']
 ['0' '0' 'O' '0']]
0100
2 1
